In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import random

import plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go

import pandas as pd
import numpy as np


from plotly.graph_objs import Figure, Layout, Bar, Table, Heatmap, Scatter, Box, Violin
from plotly.offline import init_notebook_mode, iplot


from sklearn import metrics

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
NUMBER_ACTIONS = 500
VIDEO_SIZE = 30
RAWDATASET = pd.DataFrame()
PROCDATASET = pd.DataFrame()
SEEKDATASET = pd.DataFrame()

In [4]:
plotly.offline.init_notebook_mode(connected=True)

In [5]:
names = pd.read_csv("names.csv")
RAWDATASET = pd.DataFrame(columns=["Students","Action","Age"])
PROCDATASET = pd.DataFrame(0, index= range(VIDEO_SIZE), columns=["Time","Play","Pause","Seek from","Seek to"])
for i in range(0,VIDEO_SIZE): PROCDATASET.loc[i,"Time"] = i

k = 0


In [6]:
rand_names = [names.group_name[np.random.randint(0,len(names.group_name)+1)] for n in range(0,NUMBER_ACTIONS)]
rand_names.sort()

In [7]:
for i in range(0,NUMBER_ACTIONS//5):
    playrequired = True
    currentTime = 0

    for j in range(0,5):
        timeModifierFrom = int(currentTime + np.random.triangular(0, 0 ,VIDEO_SIZE - currentTime))
        timeModifierTo = int(np.random.triangular(0,currentTime,VIDEO_SIZE))

        RAWDATASET.loc[k,"Students"] = rand_names[i]

        if(playrequired == True):
            playrequired = False
            RAWDATASET.loc[k,"Action"] = 'Play %ds'%currentTime
            lastAction = 'Play'
            PROCDATASET.loc[currentTime,"Play"] += 1

        elif(lastAction == 'Pause'):
            currentAction = random.choice(['Play', 'Seek'])

            if(currentAction == 'Play'):
                RAWDATASET.loc[k,"Action"] = 'Play %ds'%currentTime
                lastAction = 'Play'
                PROCDATASET.loc[currentTime,"Play"] += 1
            
            elif(currentAction == 'Seek'):
                RAWDATASET.loc[k,"Action"] = 'Seek from %ds to %ds'%(currentTime, timeModifierTo)
                currentTime = timeModifierTo
                lastAction = 'Seek'

        else:
            currentAction = random.choice(['Seek', 'Pause'])

            if(currentAction == 'Pause'):
                RAWDATASET.loc[k,"Action"] = 'Pause %ds'%timeModifierFrom
                currentTime = timeModifierFrom
                PROCDATASET.loc[currentTime,"Pause"] += 1
                lastAction = 'Pause'

            elif(currentAction == 'Seek'):
                RAWDATASET.loc[k,"Action"] = 'Seek from %ds to %ds'%(timeModifierFrom,timeModifierTo)
                currentTime = timeModifierTo
                PROCDATASET.loc[timeModifierFrom,"Seek from"] += 1
                PROCDATASET.loc[timeModifierTo,"Seek to"] += 1
                lastAction = 'Seek'

        k+= 1

df = PROCDATASET
df2 = df.set_index("Time")
x= df.index.get_level_values(0).values
print(x)
y=df.iloc[:,2].tolist()
print(y)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[3, 1, 4, 2, 4, 5, 1, 5, 2, 4, 10, 2, 6, 6, 11, 8, 8, 5, 7, 5, 4, 11, 8, 4, 4, 4, 2, 4, 3, 0]


In [8]:
df = PROCDATASET
trace = Table(
    header=dict(
        values=list(df.columns[:len(df.columns)]),
        fill = dict(color='#C2D4FF'),
        align = 'center'
    ),
    cells=dict(
        values=[df[i].tolist() for i in df.columns[:len(df.columns)]],
        fill = dict(color='#F5F8FF'),
        align = ['left','center']
        )
    )

data = [trace]
fig = dict(data=data)
plotly.offline.iplot(data, filename = 'pandas_table')
#print(df)


In [9]:
plotly.__version__

'3.1.0'

In [10]:
dft = PROCDATASET
df = dft.set_index("Time")
        
x = df.index.get_level_values(0).values

trace0 = dict(
    x=x,
    y=df.iloc[:,0].tolist(),
    hoverinfo='x+y',
    mode='lines',
    name='Play',
    line=dict(width=0.5,
              color='rgba(253, 174, 97, 0.75)'),
    stackgroup='one'
)
trace1 = dict(
    x=x,
    y=df.iloc[:,1].tolist(),
    hoverinfo='x+y',
    mode='lines',
    name='Pause',
    line=dict(width=0.5,
              color='rgb(184, 247, 212)'),
    stackgroup='one'
)
trace2 = dict(
    x=x,
    y=df.iloc[:,2].tolist(),
    hoverinfo='x+y',
    mode='lines',
    name='Seek from',
    line=dict(width=0.5,
              color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x=x,
    y=df.iloc[:,3].tolist(),
    hoverinfo='x+y',
    mode='lines',
    name='Seek to',
    line=dict(width=0.5,
              color='rgb(131, 90, 241)'),
    stackgroup='one'

)
data = [trace0, trace1, trace2, trace3]

fig = dict(data=data)
plotly.offline.iplot(fig, filename='stacked-area-plot-hover', validate=False)

In [24]:
x = [1]

trace1 = {
  'x': x,
  'y': [1],
  'name': 'Trace1',
  'type': 'bar'
};
trace2 = {
  'x': x,
  'y': [6],
  'name': 'Trace2',
  'type': 'bar'
};
trace3 = {
  'x': x,
  'y': [-15],
  'name': 'Trace3',
  'type': 'bar'
 }


data = [trace1, trace2, trace3];
layout = {
  'xaxis': {'title': 'X axis'},
  'yaxis': {'title': 'Y axis'},
  'barmode': 'relative',
  'title': 'Relative Barmode'
};
plotly.offline.iplot({'data': data, 'layout': layout}, filename='barmode-relative')